In [82]:
import os
from os import listdir
from os.path import isfile, join
import cv2
import numpy as np
import keras
from keras import backend as k
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense,Dropout,Activation
from keras.optimizers import Adam
#from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
import argparse
from keras.layers.normalization import BatchNormalization
#from pyimagesearch.stridednet import StridedNet
from keras import backend as K
from sklearn.preprocessing import LabelBinarizer


In [83]:
def resize(img):
    img = cv2.resize(img,(20,20))
    return img

In [84]:
X_train = []
y_train = []

# to get the name of the folder
for name_folder in os.listdir("C:\\Users\AUTOBOTS\Downloads\optical-character-recognition-OCR-master\Fnt") :

    name = 'C:\\Users\AUTOBOTS\Downloads\optical-character-recognition-OCR-master\Fnt/' + name_folder
    for f in listdir(name):
        # name of the folder is the name of the output
        y_train.append(np.asarray(name_folder))
        
        # constructing full path to the image
        name = 'C:\\Users\AUTOBOTS\Downloads\optical-character-recognition-OCR-master\Fnt/' + name_folder + '/' + f
        
        # reading the image
        image = cv2.imread(name,0)/255
        
        # appending to form the image list
        image = np.asarray(image)
        image = resize(image)
        X_train.append([image])

In [85]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_train.shape

(36560, 1, 20, 20)

In [86]:
X_train = np.reshape(X_train, [-1,20,20,1])

In [87]:
X_train.shape

(36560, 20, 20, 1)

In [88]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y_train)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y_train = onehot_encoder.fit_transform(integer_encoded)

In [92]:
class StridedNet:
    def build(width, height, depth,reg, init="he_normal"):
        # initialize the model along with the input shape to be
        # "channels last" and the channels dimension itself
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1
        
		# if we are using "channels first", update the input shape
		# and channels dimension
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1
            
# our first CONV layer will learn a total of 16 filters, each
		# Of which are 7x7 -- we'll then apply 2x2 strides to reduce
		# the spatial dimensions of the volume
        model.add(Conv2D(16, (7, 7), strides=(2, 2), padding="valid",
            kernel_initializer=init, kernel_regularizer=reg,
            input_shape=inputShape))
        
		# here we stack two CONV layers on top of each other where
		# each layerswill learn a total of 32 (3x3) filters
        model.add(Conv2D(32, (3, 3), padding="same",
            kernel_initializer=init, kernel_regularizer=reg))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(32, (3, 3), strides=(2, 2), padding="same",
            kernel_initializer=init, kernel_regularizer=reg))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Dropout(0.25))
        
        # stack two more CONV layers, keeping the size of each filter
		# as 3x3 but increasing to 64 total learned filters
        model.add(Conv2D(64, (3, 3), padding="same",
            kernel_initializer=init, kernel_regularizer=reg))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (3, 3), strides=(2, 2), padding="same",
            kernel_initializer=init, kernel_regularizer=reg))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Dropout(0.25))
        
		# increase the number of filters again, this time to 128
        model.add(Conv2D(128, (3, 3), padding="same",
            kernel_initializer=init, kernel_regularizer=reg))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(128, (3, 3), strides=(2, 2), padding="same",
            kernel_initializer=init, kernel_regularizer=reg))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Dropout(0.25))
        
        # fully-connected layer
        model.add(Flatten())
        model.add(Dense(512, kernel_initializer=init))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        
		# softmax classifier
        model.add(Dense(36, activation="softmax"))
        return model


In [100]:
#ap = argparse.ArgumentParser()
#args = vars(ap.parse_args())
    

#opt = Adam(lr=1e-4, decay=1e-4 / args[50])
model = StridedNet.build(width=20, height=20, depth=1,reg=l2(0.0005))
#model=StridedNet()
model.compile(loss="categorical_crossentropy", optimizer="Adam",metrics=["accuracy"])



In [98]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 7, 7, 16)          800       
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 7, 7, 32)          4640      
_________________________________________________________________
activation_15 (Activation)   (None, 7, 7, 32)          0         
_________________________________________________________________
batch_normalization_15 (Batc (None, 7, 7, 32)          128       
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 4, 4, 32)          9248      
_________________________________________________________________
activation_16 (Activation)   (None, 4, 4, 32)          0         
_________________________________________________________________
batch_normalization_16 (Batc (None, 4, 4, 32)         

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X_train,y_train ,test_size = .2)

In [70]:
y_train.shape

(29248, 36)

In [103]:
print(len(X_train))

36560


In [104]:
# fitting the model
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
	  width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
	  horizontal_flip=True, fill_mode="nearest")

model.fit_generator(aug.flow(X_train, y_train, batch_size=300),
                             steps_per_epoch=len(X_train) // 300,
                             epochs=50,
                             validation_data=(X_test,y_test), shuffle = 1)

Epoch 1/50
121/121 [==============================] - 14s 116ms/step - loss: 4.6651 - accuracy: 0.0485 - val_loss: 3.9540 - val_accuracy: 0.0842
Epoch 2/50
121/121 [==============================] - 14s 118ms/step - loss: 3.7472 - accuracy: 0.1089 - val_loss: 4.0082 - val_accuracy: 0.1630
Epoch 3/50
121/121 [==============================] - 17s 137ms/step - loss: 3.2648 - accuracy: 0.1768 - val_loss: 3.3554 - val_accuracy: 0.2931
Epoch 4/50
121/121 [==============================] - 15s 127ms/step - loss: 2.8992 - accuracy: 0.2503 - val_loss: 2.5281 - val_accuracy: 0.4182
Epoch 5/50
121/121 [==============================] - 17s 144ms/step - loss: 2.6106 - accuracy: 0.3208 - val_loss: 1.9728 - val_accuracy: 0.5161
Epoch 6/50
121/121 [==============================] - 18s 146ms/step - loss: 2.3963 - accuracy: 0.3766 - val_loss: 1.9694 - val_accuracy: 0.5070
Epoch 7/50
121/121 [==============================] - 17s 144ms/step - loss: 2.2078 - accuracy: 0.4275 - val_loss: 1.7079 - val_ac

In [105]:
#saving the model 
model.save('C:\\Users\AUTOBOTS\Downloads\optical-character-recognition-OCR-master\models\model4.h5')

In [106]:
np.save('C:\\Users\AUTOBOTS\Downloads\optical-character-recognition-OCR-master\models/xtrain',X_train,allow_pickle=True)